## InfluxDB ##

In [1]:
from influxdb import InfluxDBClient
import json

In [2]:
client = InfluxDBClient(host='influxdb-influxdb.default.svc.cluster.local', port=8086)

In [3]:
dbs= client.get_list_database()

In [4]:
client.switch_database('prices')

In [5]:
measurements = client.get_list_measurements()

In [6]:
results = client.query('select * from BTC_bitcoin ORDER by time DESC LIMIT 1')
results.raw

{'statement_id': 0,
 'series': [{'name': 'BTC_bitcoin',
   'columns': ['time',
    'marketcap_usd',
    'price_btc',
    'price_usd',
    'volume_usd'],
   'values': [['2018-11-01T12:39:15Z',
     110849407485,
     1,
     6387.6526323,
     4201408489]]}]}

In [7]:
str = str(measurements)

In [8]:
str = str.replace("\'", "\"")

In [9]:
raw_input = json.loads(str)

In [10]:
output_dict = [x for x in raw_input if x['name'] == '611_USD']

In [11]:
output_dict

[{'name': '611_USD'}]

In [12]:
clear_USD = [x for x in measurements 
               if x['name'].endswith('_USD') == False
              ]

In [13]:
clear_BTC = [x for x in clear_USD 
               if x['name'].endswith('_BTC') == False
              ]

In [14]:
clear_BTC1 = [x for x in clear_BTC 
               if x['name'].endswith('coin') 
              ]

In [14]:
for x in output_dict:
    results = client.query(('SELECT * from "%s" WHERE time >= now() - 1d LIMIT 100') %  x['name'])
    points = list(results.get_points())
    #points[0]['name'] = x['name']
    for p in points:
        p['name'] = x['name']
    #print(points)
      


In [14]:
points

[{'time': '2018-10-30T21:02:16Z',
  'marketcap_usd': 109824690991,
  'price_btc': 1,
  'price_usd': 6329.86216286,
  'volume_usd': 3759406473,
  'name': 'BTC_bitcoin'},
 {'time': '2018-10-30T21:04:00Z',
  'marketcap_usd': 109834543107,
  'price_btc': 1,
  'price_usd': 6330.43,
  'volume_usd': 3760800000,
  'name': 'BTC_bitcoin'},
 {'time': '2018-10-30T21:07:13Z',
  'marketcap_usd': 109884758732,
  'price_btc': 1,
  'price_usd': 6333.32423062,
  'volume_usd': 3764836824,
  'name': 'BTC_bitcoin'},
 {'time': '2018-10-30T21:09:00Z',
  'marketcap_usd': 109839748182,
  'price_btc': 1,
  'price_usd': 6330.73,
  'volume_usd': 3758690000,
  'name': 'BTC_bitcoin'},
 {'time': '2018-10-30T21:11:15Z',
  'marketcap_usd': 109900786607,
  'price_btc': 1,
  'price_usd': 6334.24801412,
  'volume_usd': 3763869558,
  'name': 'BTC_bitcoin'},
 {'time': '2018-10-30T21:14:00Z',
  'marketcap_usd': 109909149182,
  'price_btc': 1,
  'price_usd': 6334.73,
  'volume_usd': 3764020000,
  'name': 'BTC_bitcoin'},
 {'t

In [14]:
import datetime
import pandas as pd

In [12]:
d = '2018-10-29'
x= 'BTC_bitcoin'

query = """SELECT * from "%s" WHERE time >= '%s' LIMIT 100""" %  (x, d )
print(query)

for x in output_dict:
    results = client.query(query)
    points = list(results.get_points())

    #print(date)
points

SELECT * from "BTC_bitcoin" WHERE time >= '2018-10-29' LIMIT 100


[{'time': '2018-10-29T00:03:11Z',
  'marketcap_usd': 112621848426,
  'price_btc': 1,
  'price_usd': 6492.3515466,
  'volume_usd': 3414332000},
 {'time': '2018-10-29T00:04:00Z',
  'marketcap_usd': 112621821597,
  'price_btc': 1,
  'price_usd': 6492.35,
  'volume_usd': 3414330000},
 {'time': '2018-10-29T00:08:16Z',
  'marketcap_usd': 112437622356,
  'price_btc': 1,
  'price_usd': 6481.73140114,
  'volume_usd': 3417639900},
 {'time': '2018-10-29T00:09:00Z',
  'marketcap_usd': 112437598050,
  'price_btc': 1,
  'price_usd': 6481.73,
  'volume_usd': 3417640000},
 {'time': '2018-10-29T00:14:00Z',
  'marketcap_usd': 112546709737,
  'price_btc': 1,
  'price_usd': 6488.02,
  'volume_usd': 3430450000},
 {'time': '2018-10-29T00:15:12Z',
  'marketcap_usd': 112538311527,
  'price_btc': 1,
  'price_usd': 6487.53586539,
  'volume_usd': 3432536937},
 {'time': '2018-10-29T00:19:00Z',
  'marketcap_usd': 112482677139,
  'price_btc': 1,
  'price_usd': 6484.31,
  'volume_usd': 3434010000},
 {'time': '2018-1

In [27]:
date = '2018-10-29 12:00'

data_range=pd.date_range(date, datetime.datetime.today())

for d in data_range:
    #print(d)
    d=d.strftime('%Y-%m-%d %H:%M')
    print(d)

2018-10-29 12:00
2018-10-30 12:00
2018-10-31 12:00
2018-11-01 12:00


In [26]:
date = '2018-10-30'

data_range=pd.date_range(date, datetime.datetime.today())

for d in data_range:
    #print(d)
    d=d.strftime('%Y-%m-%d')
    print(d)
    
    for x in output_dict:
        query = """SELECT * from "%s" WHERE time >= '%s' LIMIT 100""" %  (x['name'], d )
        print(query)
        results = client.query(query)
        points = list(results.get_points())
        for i, p in enumerate(points):
            #print('index %d, value %s' % (i, p)) 
            points[i]['name'] = x['name']
            print(points[i])
            #producer.send_messages('test', json.dumps(points[i], default=json_util.default).encode('utf-8'))


2018-10-30
SELECT * from "611_USD" WHERE time >= '2018-10-30' LIMIT 100
{'time': '2018-10-30T00:00:43Z', 'marketcap': 50207, 'price': 0.1091232534, 'volume': 4, 'name': '611_USD'}
{'time': '2018-10-30T00:05:28Z', 'marketcap': 50336, 'price': 0.1094035678, 'volume': 4, 'name': '611_USD'}
{'time': '2018-10-30T00:13:36Z', 'marketcap': 50203, 'price': 0.1091157294, 'volume': 4, 'name': '611_USD'}
{'time': '2018-10-30T00:18:32Z', 'marketcap': 50202, 'price': 0.1091114545, 'volume': 4, 'name': '611_USD'}
{'time': '2018-10-30T00:22:19Z', 'marketcap': 50365, 'price': 0.1094671616, 'volume': 4, 'name': '611_USD'}
{'time': '2018-10-30T00:28:28Z', 'marketcap': 50208, 'price': 0.1091261264, 'volume': 4, 'name': '611_USD'}
{'time': '2018-10-30T00:35:33Z', 'marketcap': 50356, 'price': 0.1094472379, 'volume': 4, 'name': '611_USD'}
{'time': '2018-10-30T00:41:33Z', 'marketcap': 50215, 'price': 0.109141121, 'volume': 4, 'name': '611_USD'}
{'time': '2018-10-30T00:45:27Z', 'marketcap': 50353, 'price': 0.1

{'time': '2018-10-31T00:01:44Z', 'marketcap': 78123, 'price': 0.1697671498, 'volume': 518, 'name': '611_USD'}
{'time': '2018-10-31T00:06:18Z', 'marketcap': 78123, 'price': 0.1697671498, 'volume': 518, 'name': '611_USD'}
{'time': '2018-10-31T00:13:14Z', 'marketcap': 78426, 'price': 0.1704252263, 'volume': 520, 'name': '611_USD'}
{'time': '2018-10-31T00:18:38Z', 'marketcap': 78426, 'price': 0.1704252263, 'volume': 520, 'name': '611_USD'}
{'time': '2018-10-31T00:24:41Z', 'marketcap': 78413, 'price': 0.1703981996, 'volume': 520, 'name': '611_USD'}
{'time': '2018-10-31T00:29:28Z', 'marketcap': 78147, 'price': 0.1698194185, 'volume': 518, 'name': '611_USD'}
{'time': '2018-10-31T00:37:34Z', 'marketcap': 78465, 'price': 0.1705090708, 'volume': 520, 'name': '611_USD'}
{'time': '2018-10-31T00:41:38Z', 'marketcap': 78204, 'price': 0.1699428912, 'volume': 518, 'name': '611_USD'}
{'time': '2018-10-31T00:46:39Z', 'marketcap': 78500, 'price': 0.1705846598, 'volume': 520, 'name': '611_USD'}
{'time': '

{'time': '2018-11-01T00:00:25Z', 'marketcap': 64776, 'price': 0.1407384318, 'volume': 145, 'name': '611_USD'}
{'time': '2018-11-01T00:04:43Z', 'marketcap': 64776, 'price': 0.1407384318, 'volume': 145, 'name': '611_USD'}
{'time': '2018-11-01T00:09:23Z', 'marketcap': 64776, 'price': 0.1407384318, 'volume': 145, 'name': '611_USD'}
{'time': '2018-11-01T00:17:37Z', 'marketcap': 64776, 'price': 0.1407384318, 'volume': 145, 'name': '611_USD'}
{'time': '2018-11-01T00:22:44Z', 'marketcap': 64776, 'price': 0.1407384318, 'volume': 145, 'name': '611_USD'}
{'time': '2018-11-01T00:27:13Z', 'marketcap': 64776, 'price': 0.1407384318, 'volume': 145, 'name': '611_USD'}
{'time': '2018-11-01T00:33:44Z', 'marketcap': 64776, 'price': 0.1407384318, 'volume': 145, 'name': '611_USD'}
{'time': '2018-11-01T00:40:37Z', 'marketcap': 64776, 'price': 0.1407384318, 'volume': 145, 'name': '611_USD'}
{'time': '2018-11-01T00:45:18Z', 'marketcap': 64776, 'price': 0.1407384318, 'volume': 145, 'name': '611_USD'}
{'time': '

In [67]:
print ("Length : %d" % len (points))

Length : 10000


In [68]:
points[0]

{'time': '2013-04-28T18:47:21Z',
 'marketcap_usd': 1500517590,
 'price_btc': 1,
 'price_usd': 135.3,
 'volume_usd': 0,
 'name': 'BTC_bitcoin'}

In [15]:
out = json.dumps(results.raw)
print(out)

{"statement_id": 0, "series": [{"name": "ALC_allcoin", "columns": ["time", "price_btc", "price_usd"], "values": [["2018-10-28T19:25:54Z", 2.28e-06, 0.01474555]]}]}


In [28]:
print(output_dict)

[{'name': '611_USD'}]


## Working with JSON output ##

In [21]:
for x in output_dict:
    results = client.query(('SELECT * from "%s" ORDER by time DESC LIMIT 1') %  x['name'])
    
    points = list(results.get_points())
    print(points)
    data_file = x['name']+'.json'
    with open(data_file, 'w') as outfile:
        #json.dump(points, outfile)
        producer.send_messages('test', json.dumps(points, default=json_util.default).encode('utf-8'))
        print(x['name']+' message sent')
# 611_USD.json
#results.raw
#print(points)

[{'time': '2018-10-28T19:25:38Z', 'marketcap': 58289, 'price': 0.1267165628, 'volume': 23}]
611_USD message sent


In [37]:
results.raw    

{'statement_id': 0,
 'series': [{'name': '611_USD',
   'columns': ['time', 'marketcap', 'price', 'volume'],
   'values': [['2018-10-22T19:35:33Z', 65116, 0.1417095519, 122]]}]}

In [188]:
# save relusts.raw in a json file
data_file = output_dict['name']
with open('data.json', 'w') as outfile:
    json.dump(results.raw, outfile)

In [38]:
# save columns and values in a json file
with open('data.json', 'w') as outfile:
    json.dump(points, outfile)

In [131]:
def read_data(measurements):
    client = InfluxDBClient(host='influxdb-influxdb.default.svc.cluster.local', port=8086)
    client.switch_database('prices')
    
    for x in measurements:
        results = client.query(('SELECT * from "%s" ORDER by time DESC LIMIT 1') %  x['name'])
        points = list(results.get_points())
        #params = points[0]
        #params['name'] = x['name']
        
        
        points[0]['name'] = x['name']
        #producer.send_messages('test', json.dumps(points[0], default=json_util.default).encode('utf-8'))

        

        #data_file = x['name']+'.json'
        #with open(data_file, 'w') as outfile:
            #json.dump(points, outfile)
    return points[0]
    #return params

In [132]:
read_data(output_dict)


{'time': '2018-10-28T20:58:37Z',
 'marketcap': 57960,
 'price': 0.1259996609,
 'volume': 23,
 'name': '611_USD'}

In [118]:
type(x)

NoneType

In [106]:
print(x.keys)

<built-in method keys of dict object at 0x0128BDE0>


In [61]:
json.dumps(x)

'[{"time": "2018-10-28T20:07:33Z", "marketcap": 57928, "price": 0.1259316913, "volume": 23, "name": "611_USD"}]'

## Kafka ##

In [25]:
from kafka import SimpleProducer, KafkaClient


kafka = KafkaClient('blockchain-kafka-kafka.default.svc.cluster.local:9092')
producer = SimpleProducer(kafka)
#producer.send_messages('test', b'async message')

In [26]:
import yaml
import json
from bson import json_util


In [27]:
import kafka as k
k.__version__

'1.4.3'

In [213]:
with open('611_USD.json') as f:
    data = json.load(f)

In [214]:
print(data)

[{'time': '2018-10-28T19:45:28Z', 'marketcap': 57956, 'price': 0.1259925901, 'volume': 23}]


In [22]:
with open('611_USD.json') as f:
    for line in f:
        d = yaml.safe_load(line)
        jd = json.dumps(d)
       
        producer.send_messages('test', json.dumps(jd, default=json_util.default).encode('utf-8'))


In [8]:
# send 10 messages to kafka
for i in range(10):
    data = { 'tag ': 'blah',
        'name' : 'sam',
        'index' : i,
        'score': 
            {'row1': 100,
             'row2': 200
        }
    }   
    producer.send_messages('test', json.dumps(data, indent=4, default=json_util.default).encode('utf-8'))

In [ ]:
for x in output_dict:
    results = client.query(('SELECT * from "%s" ORDER by time DESC LIMIT 1') %  x['name'])
    
    points = list(results.get_points())
    print(points)
    data_file = x['name']+'.json'
    with open(data_file, 'w') as outfile:
        #json.dump(points, outfile)
        #producer.send_messages('test', json.dumps(points, default=json_util.default).encode('utf-8'))
        print(x['name']+' message sent')

## Pandas 

In [20]:
import pandas as pd
import datetime 


In [173]:
startdate = '2009/11/23'
enddate = '2009/12/23'

In [199]:
#pd.to_datetime(['2005/11/23', pd.Timestamp.now])
list=pd.DatetimeIndex(['2005-11-23', '2010-12-31'], dtype='datetime64[ns]', freq=None)

In [183]:
datetime.datetime.now()

datetime.datetime(2018, 10, 31, 22, 26, 34, 420232)

In [33]:
li=pd.date_range('2018/10/23', datetime.datetime.today())

In [34]:
li

DatetimeIndex(['2018-10-23', '2018-10-24', '2018-10-25', '2018-10-26',
               '2018-10-27', '2018-10-28', '2018-10-29', '2018-10-30',
               '2018-10-31', '2018-11-01'],
              dtype='datetime64[ns]', freq='D')

In [48]:
[d.strftime('%Y%m%d') for d in pd.date_range('20130226','20130302')]:
    print(d)

SyntaxError: invalid syntax (<ipython-input-48-af58944084ac>, line 1)

In [49]:
for l in list:
    print(l.strftime('%Y-%m-%d'))

2018-10-23
2018-10-24
2018-10-25
2018-10-26
2018-10-27
2018-10-28
2018-10-29
2018-10-30
2018-10-31


In [197]:
pd.DataFrame(data=list)

,0


In [89]:
last_index = df.last_valid_index()

In [91]:
last = list[last_index]

In [158]:
def forward_date_range(start_dt, end_dt, span_days):
    #
    # Generate tuples with intervals from given range of dates (forward)
    #
    # forward_date_range('2012-01-01', '2012-01-5', 2)
    #
    # 1st yield = ('2012-01-01', '2012-01-03')
    # 2nd yield = ('2012-01-04', '2012-01-05')
    start_dt = datetime.datetime.strptime(start_dt, '%Y-%m-%d')
    end_dt   = datetime.datetime.strptime(end_dt, '%Y-%m-%d')
    span     = datetime.timedelta(days=span_days)
    step     = datetime.timedelta(days=1)

    while start_dt + span < end_dt:
        current = start_dt + span
        yield start_dt.strftime('%Y-%m-%d'), current.strftime('%Y-%m-%d')
        start_dt = current + step
    else:
        yield start_dt.strftime('%Y-%m-%d'), end_dt.strftime('%Y-%m-%d')

In [163]:
f=forward_date_range('2018-01-01','2018-10-30', 30)

In [172]:
l = next(f)
for x  in l:
    print(type(x))
    


<class 'str'>
<class 'str'>


In [174]:
df[df.some_date.between(start_date, end_date)]

AttributeError: 'DataFrame' object has no attribute 'some_date'